#### [Try-7]
- SJY_LGBM_upgrade7.ipynb
- 이상치 제거(IQR), Optuna, KFold 
- LGBM, Extra Trees Regressor(기본모델) & 선형회귀(메타모델)-> stacking ensemble
- Average RMSE:  11416.153423597494 (크게 향상)
- Average MSE:  130559586.11656937 (감퇴)

In [1]:
!pip install optuna
!pip install lightgbm
!apt-get install -y fonts-nanum
!pip install mlxtend 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 26 not upgraded.


In [2]:
# 시각화
import matplotlib.pyplot as plt

# 유틸리티
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

# 모델
import optuna
import lightgbm
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor

In [3]:
# 필요한 데이터를 load
total_path = "../total-data_ver2.csv"

dt_total = pd.read_csv(total_path)

# is_test 컬럼 값이 0인 행을 train 데이터셋으로 저장
df_train = dt_total[dt_total['is_test'] == 0]

# is_test 컬럼 값이 1인 행을 test 데이터셋으로 저장
df_test = dt_total[dt_total['is_test'] == 1]

In [4]:
# '계약년월'에서 '연도' 추출 후 int 형으로 변환하여 새로운 열 추가
df_train = df_train.assign(year=df_train['계약년월'].astype('str').str[:4].astype(int)).query('year > 2020')

# '계약년월'에서 '월' 추출 후 5로 나눈 값을 새로운 열로 추가
df_train, df_test = map(lambda df: df.assign(계약월=lambda x: (x['계약년월'] % 100) // 5), [df_train, df_test])

# 타겟 변수 생성
df_train['price'] = df_train.groupby(['아파트명', '계약월'])['target'].transform('mean')

# 중복 데이터 제거
df_train = df_train.drop_duplicates(subset=['아파트명', '계약월'])

In [5]:
def remove_outliers_iqr(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR  # Modify here
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return df

# 전용면적에 대한 이상치 제거
df_train = remove_outliers_iqr(df_train, '전용면적')

In [6]:
# 특징 변수와 타겟 변수 설정
X = df_train[['전용면적', 'x']]
y = df_train['price']
test = df_test[['전용면적', 'x']]

In [7]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
    }

    model = lightgbm.LGBMRegressor(**params)
    kf = KFold(n_splits=5, shuffle=True, random_state=2023)
    rmse_list = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        val_pred = model.predict(X_val)
        rmse = sqrt(mean_squared_error(y_val, val_pred))
        rmse_list.append(rmse)

    return np.mean(rmse_list)

In [8]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-01-24 03:11:02,504] A new study created in memory with name: no-name-f84027e8-844f-4fb9-806f-97d856cef884


/tmp/ipykernel_600429/3468174135.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000101 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7226, number of used features: 2
[LightGBM] [Info] Start training from score 91770.722830
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[I 2024-01-24 03:11:03,069] Trial 0 finished with value: 57651.92673446267 and parameters: {'n_estimators': 183, 'max_depth': 5, 'learning_rate': 0.0010997925738917658}. Best is trial 0 with value: 57651.92673446267.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7227, number of used features: 2
[LightGBM] [Info] Start training from score 91640.817805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

...

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7226, number of used features: 2
[LightGBM] [Info] Start training from score 91770.722830
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

/tmp/ipykernel_600429/3468174135.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7226, number of used features: 2
[LightGBM] [Info] Start training from score 91807.021091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[I 2024-01-24 03:14:11,864] Trial 99 finished with value: 45058.592874308066 and parameters: {'n_estimators': 820, 'max_depth': 13, 'learning_rate': 0.12213090975077548}. Best is trial 83 with value: 44882.889546308324.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [9]:
print('Best parameters: ', study.best_params)

Best parameters:  {'n_estimators': 855, 'max_depth': 18, 'learning_rate': 0.14781024970890544}


In [10]:
# 기본 모델 정의
lgbm_model = lightgbm.LGBMRegressor(**study.best_params)
et_model = ExtraTreesRegressor(n_estimators=100, random_state=2023)
lr_model = LinearRegression()  # 메타 모델로 선형 회귀 사용

In [11]:
# 스태킹 모델 정의
stacking_model = StackingCVRegressor(regressors=[lgbm_model, et_model],  # 기본 모델로 LightGBM, Extra Trees 사용
                                     meta_regressor=lr_model,  # 메타 모델로 선형 회귀 사용
                                     random_state=2023)

In [12]:
# 모델 학습
stacking_model.fit(X.values, y.values)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 7226, number of used features: 2
[LightGBM] [Info] Start training from score 91770.722830
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set 

StackingCVRegressor(meta_regressor=LinearRegression(), random_state=2023,
                    regressors=[LGBMRegressor(learning_rate=0.14781024970890544,
                                              max_depth=18, n_estimators=855),
                                ExtraTreesRegressor(random_state=2023)])

In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state=2023)
rmse_list = []
mse_list = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # 스태킹 모델로 예측
    val_pred = stacking_model.predict(X_val.values)

    rmse = sqrt(mean_squared_error(y_val, val_pred))
    rmse_list.append(rmse)

    mse = mean_squared_error(y_val, val_pred)
    mse_list.append(mse)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [14]:
print("Average RMSE: ", np.mean(rmse_list))

Average RMSE:  11416.153423597494


In [15]:
print("Average MSE: ", np.mean(mse_list))

Average MSE:  130559586.11656937


In [16]:
# 스태킹 모델로 최종 예측
pred = stacking_model.predict(test.values)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [17]:
# 예측 결과 저장
pd.DataFrame(pred.astype(int), columns=['target']).to_csv('simple_upgrade7.csv', index=False)

# 저장된 파일 확인
print(pd.read_csv('simple_upgrade7.csv'))

      target
0      67001
1      56622
2      80451
3      39950
4      39950
...      ...
9267  102823
9268  102823
9269  111609
9270   62854
9271   62854

[9272 rows x 1 columns]
